In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


llm = ChatOpenAI(
    temperature=0.3,
    # model="gpt-4",
    )


def login_tool(login_url: str, id: str, pwd: str, btn_name: str):
    chrome_options =  webdriver.ChromeOptions()
    chrome_options.add_experimental_option("detach", True)
    
    global driver

    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(3)

    # 로그인 페이지로 이동
    driver.get(login_url)

    # 로그인 폼 요소 찾기
    id_input = driver.find_element(By.XPATH,'//*[@id="id"]')
    pwd_input = driver.find_element(By.XPATH, '//*[@id="password"]')

    # 사용자 이름과 비밀번호 입력
    id_input.send_keys(id)
    pwd_input.send_keys(pwd)

    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{btn_name}')]"))
    )
    element.click()
    return driver


# 마타와 XX 선택할 때
def first_tool(btn_name: str):
    if "마타와" in btn_name:
        cond = f"//a[.//h3[contains(., '{btn_name}')]]"
    elif "[마타와" in btn_name:
        cond = f"//a[contains(., '{btn_name}')]"
    else:
        cond = f"//button[contains(., '{btn_name}')]"
        
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, cond))
    )
    element.click()
    return driver


# STEP 01. 학년 선택하고 다음으로 이동
def second_tool(click_btn, move_btn):
    element = driver.find_elements(By.XPATH, f"//p[contains(., '{click_btn}')]")
    element[0].click()

    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{move_btn}')]"))
    )
    element.click()
    return driver


def third_tool(select_box_btn, grade_btn, selected_class_list, move_btn):
    placeholder = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//div[contains(text(), '{select_box_btn}')]"))
    )
    placeholder.click()

    grade_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//div[text()='{grade_btn}']"))
    )
    grade_element.click()

    # TODO: 복수 반 선택 로직
    for sc in selected_class_list:
        class_element = driver.find_element(By.XPATH, f"//div[contains(text(), '{sc}')]")
        class_element.click()
    
    move_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{move_btn}')]"))
    )
    move_element.click()
    return driver


agent = initialize_agent(
    llm=llm, 
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=login_tool,
            name="login tool",
            description="""
            With this tool, you can access a url where login is possible, enter the id and password(pwd), and by pressing a button, you can log in to the page.
            """,
        ), 
        StructuredTool.from_function(
            func=first_tool,
            name="first tool",
            description="""
            This tool is used simply for the action of clicking a single button.
            """,
        ),
        StructuredTool.from_function(
            func=second_tool,
            name="second tool",
            description="""
            This tool is used to select one condition and then move to the next page through a '다음' or '완료' button. A total of two clicks are involved.
            Use this tool in the final step of submission.
            """,
        ),
        StructuredTool.from_function(
            func=third_tool,
            name="third tool",
            description="""
            This tool allows you to first select a target through a placeholder, and then choose various conditions. Therefore, more than three clicks will occur.
            """,
        ),
    ],   
)


prompt = """ 

""" 
agent.invoke(prompt)



> Entering new AgentExecutor chain...
Question: Should I perform all the steps mentioned in the given order?
Thought: Yes, I should perform all the steps mentioned in the given order to complete the task.
Action:
```json
{
  "action": "login tool",
  "action_input": {
    "login_url": "https://m5-dev.matamath.net/test.es/login",
    "id": "test",
    "pwd": "2023ejrmffhfl!",
    "btn_name": "로그인"
  }
}
```


The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\USER\AppData\Local\Programs\Python\Python311\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.130); currently, chromedriver 120.0.6099.109 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry



Observation: <selenium.webdriver.chrome.webdriver.WebDriver (session="8715131f3c781cf6d7e8ca2928badf48")>
Thought:I have successfully logged in. Now I need to click on the '마타와 개념학습' button.
Action:
```json
{
  "action": "first tool",
  "action_input": {
    "btn_name": "마타와 개념학습"
  }
}
```

Observation: <selenium.webdriver.chrome.webdriver.WebDriver (session="8715131f3c781cf6d7e8ca2928badf48")>
Thought:I have successfully clicked on the '마타와 개념학습' button. Now I need to click on '초3-2' and then click on the '다음' button to proceed.
Action:
```json
{
  "action": "second tool",
  "action_input": {
    "click_btn": "초3-2",
    "move_btn": "다음"
  }
}
```

Observation: <selenium.webdriver.chrome.webdriver.WebDriver (session="8715131f3c781cf6d7e8ca2928badf48")>
Thought:I have successfully clicked on '초3-2' and then clicked on the '다음' button to proceed. Now I need to select '10강' and click on the '다음' button again.
Action:
```json
{
  "action": "second tool",
  "action_input": {
    "click_b

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (145, 878)
  (Session info: chrome=120.0.6099.130)
Stacktrace:
	GetHandleVerifier [0x00007FF67AF982B2+55298]
	(No symbol) [0x00007FF67AF05E02]
	(No symbol) [0x00007FF67ADC05AB]
	(No symbol) [0x00007FF67AE07A77]
	(No symbol) [0x00007FF67AE05E39]
	(No symbol) [0x00007FF67AE03C08]
	(No symbol) [0x00007FF67AE02C8A]
	(No symbol) [0x00007FF67ADF87BF]
	(No symbol) [0x00007FF67AE220AA]
	(No symbol) [0x00007FF67ADF80CF]
	(No symbol) [0x00007FF67AE222C0]
	(No symbol) [0x00007FF67AE3AAA4]
	(No symbol) [0x00007FF67AE21E83]
	(No symbol) [0x00007FF67ADF670A]
	(No symbol) [0x00007FF67ADF7964]
	GetHandleVerifier [0x00007FF67B310AAB+3694587]
	GetHandleVerifier [0x00007FF67B36728E+4048862]
	GetHandleVerifier [0x00007FF67B35F173+4015811]
	GetHandleVerifier [0x00007FF67B0347D6+695590]
	(No symbol) [0x00007FF67AF10CE8]
	(No symbol) [0x00007FF67AF0CF34]
	(No symbol) [0x00007FF67AF0D062]
	(No symbol) [0x00

In [10]:
from selenium import webdriver

# 웹 드라이버 경로 설정
# driver_path = "웹 드라이버 경로"

# 웹 드라이버 실행
driver = webdriver.Chrome()

# HTML 파일 읽어오기
file_path = "HTML 파일 경로"
with open(file_path, "r") as file:
    html = file.read()

# 웹 드라이버로 HTML 파일 열기
driver.get("data:text/html;charset=utf-8," + html)

# 중1(상) 학습 범위 선택
element = driver.find_element_by_xpath("//input[@value='중1(상)']")
element.click()

# 실행 버튼 클릭
button = driver.find_element_by_xpath("//button[@id='execute-button']")
button.click()

# 웹 드라이버 종료
driver.quit()

The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\USER\AppData\Local\Programs\Python\Python311\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.130); currently, chromedriver 120.0.6099.109 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


FileNotFoundError: [Errno 2] No such file or directory: 'HTML 파일 경로'